In this notebook, we use the [method of manufactured solutions](http://prod.sandia.gov/techlib/access-control.cgi/2000/001444.pdf) to verify that [Phaseflow](https://github.com/geo-fluid-dynamics/phaseflow-fenics) correctly solves its governing equations.

# Set up this Jupyter notebook
Enable equation numbering:

In [1]:
%%javascript
MathJax.Hub.Config({
  TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

Enable inline plotting with matplotlib:

In [2]:
%matplotlib inline

# Set up SymPy

In [3]:
import sympy
from sympy.physics.vector import ReferenceFrame
from sympy.physics.vector import divergence as div, gradient as grad


t = sympy.symbols('t')

R = ReferenceFrame('R')

diff, exp, sin, tanh, transpose = sympy.diff, sympy.exp, sympy.sin, sympy.tanh, sympy.transpose

x, y = R[0], R[1]

# Define the governing equations
The governing equations are

$$\begin{align}
    \nabla\cdot\mathbf{u} &= s_p, \\
    \frac{\partial}{\partial t}\mathbf{u} + \nabla p - \nabla\cdot\left(2 \mu_{SL}\left((P(T)\right) \mathbf{D(u)}\right) 
    + \mathbf{f}_B(T) &= \mathbf{s_u}, \\
    \frac{\partial}{\partial t}(CT) + \nabla\cdot(CT\mathbf{u}) 
    - \mathscr{Pr}^{-1}\nabla\cdot(K\nabla T) + 
    \mathscr{Ste}^{-1} \frac{\partial}{\partial t} P(T) &= s_T
\end{align}$$

where

$$
\begin{align}
    \mathbf{D(u)} &= \frac{1}{2}\left(\nabla\mathbf{u} + \nabla\mathbf{u}^\intercal\right),\\
    P(T;T_f,r) &= \frac{1}{2}\left(1 - \tanh\frac{T_f - T}{r}\right),\\
    \mu_{SL}(P(T)) &= \mu_S + \left(\mu_L - \mu_S\right)P(T)
\end{align}
$$

In [4]:
def D(u):
    
    return 0.5*(grad(u, R) + transpose(u, R))
    

def P_general(T, T_f):
    
    return 0.5*(1. - tanh((T_f - T)/r))


def mu_SL(PofT):
    
    return mu_S + (mu_L - mu_S)*PofT

though for this study we will only consider $T_f = 0.01$.

In [5]:
T_f = 0.01

def P(T):
    
    return P_general(T, T_f)

Typically the right-hand sides are $\mathbf{s} = \left( s_p, \mathbf{s_u}, s_T \right)^\intercal = \mathbf{0}$, but we include $\mathbf{s}$ as a generic source term to facilitate code verification via MMS.

We denote the vector-valued solution of this system of PDE's as
$$\begin{align}
    \mathbf{w} = (p, \mathbf{u}, T)^\intercal
\end{align}$$

# Regularize
Note that we have already regularized the phase function with a hyperbolic tangent function. While testing Phaseflow so far, small smoothing parameters $r$ have destabilized the Newton solver. For the current MMS verification, we will use $r = 0.025$, the value from [the toy PCM melting regression test](https://github.com/geo-fluid-dynamics/phaseflow-fenics/blob/master/tests/test_melt_pcm.py).

In [6]:
r = 0.025

# Select a buoyancy model
For this study, we will only consider the linear buoyancy model from Danaila's simulation of the octadecane PCM melting benchmark, i.e.
$$ \begin{align}
    \mathbf{f}_B(T) = \left(0, \frac{-\mathscr{Ra}}{\mathscr{PrRe}^2}T\right)^\intercal
\end{align}$$

In [7]:
def f_B(T, Ra, Pr, Re):
    
    return -Ra/(Pr*Re*Re)*T*R.y

# Manufacture solution

To test the full capabilities of the implementation, we must select a solution where $\mathbf{u}$ and $T$ are twice differentiable in space and once differentiable in time, and where $p$ is once differentiable in space. For the present study, we will only consider a two-dimensional solution.

$$
\begin{align}
    p_\mathrm{MMS} &= e^{t} (\sin x + \sin{2y}), \\
    \mathbf{u}_\mathrm{MMS} &= e^{2t} (\sin{3x} + \sin{4y}, \sin{5x} + \sin{6y})^\intercal, \\
    T_\mathrm{MMS} &= e^{3t} (\sin{7x} + \sin{8y})
\end{align}
$$

In [8]:
p, u, T = exp(t)*(sin(x) + sin(2.*y)),\
    exp(2.*t)*((sin(3.*x) + sin(4.*y))*R.x + (sin(5.*x) + sin(6.*y))*R.y),\
    exp(3.*t)*(sin(7.*x) + sin(8.*y))
    
# @todo Re-design the manufactured temperature solution such that it contains T_f
    
print("p = " + str(p))

print("u = " + str(u))

print("T = " + str(T))

p = (sin(R_x) + sin(2.0*R_y))*exp(t)
u = (sin(3.0*R_x) + sin(4.0*R_y))*exp(2.0*t)*R.x + (sin(5.0*R_x) + sin(6.0*R_y))*exp(2.0*t)*R.y
T = (sin(7.0*R_x) + sin(8.0*R_y))*exp(3.0*t)


# Derive the manufactured source term

Given the manufactured solution $\mathbf{w}_{\mathrm{MMS}}$, derive the manufactured source terms $\mathbf{s}_{\mathrm{MMS}}$ with SymPy.

In [9]:
s_p = div(u, R)

print("s_p = " + str(s_p))

s_u = diff(u, t) 

s_u += grad(p, R)

s_u -= div(2.*mu_SL(P(T))*D(u), R) 

s_u += f_B(T)

print("s_u = " + str(s_u))

s_p = 3.0*exp(2.0*t)*cos(3.0*R_x) + 6.0*exp(2.0*t)*cos(6.0*R_y)


AttributeError: 'Symbol' object has no attribute 'x'

In [ ]:
import fenics

In [ ]:
import phaseflow